In [1]:
import pandas as pd
from bokeh.models.sources import ColumnDataSource

    cd Desktop
    activate nyc-buildings
    jupyter notebook

In [2]:
# %ls

In [3]:
mepl = pd.read_pickle('most_enumerate_private_landowners.p')
wpl = pd.read_pickle('wealthiest_private_landowners.p')
lpl = pd.read_pickle('largest_private_landowners.p')
# lpl = ColumnDataSource(data=pd.read_pickle('largest_private_landowners.p'))

In [4]:
# pd.read_csv("nyc_building_sales.csv")

In [5]:
# pd.read_pickle('most_enumerate_private_landowners.p').to_excel("most_enumerate_private_landowners.xls")
# pd.read_pickle('wealthiest_private_landowners.p').to_excel("wealthiest_private_landowners.xls")
# pd.read_pickle('largest_private_landowners.p').to_excel("largest_private_landowners.xls")

In [6]:
# ls

In [7]:
from ast import literal_eval

In [8]:
#for (address, (latitude, longitude)) in columbia_addresses.items():
 #   print(address, latitude, longitude)

In [9]:
mepl[['Owner', 'Addresses']]

,Owner,Addresses
Rank,,
1,Trustees Of Columbia,209
2,Duestche Bank National Trust,86
3,Spring Creek Nehemaih,81
4,Northeastern Conference of Seventh-Day Adventists,79
5,New York University,77
6,U.S. Bank National Association,77
7,Consolidated Edison Company,74
8,Maguire Avenue Realty,71
9,New York Garden Trust,62


In [10]:
wpl[['Owner', 'Total Value']]

,Owner,Total Value
Rank,,
1,New York University,2.260595e+09
2,RCPI Landmark Properties LLC,1.923597e+09
3,Trustees Of Columbia,1.790191e+09
4,One Bryant Park,1.715132e+09
5,767 Fifth Partnerns LLC,1.703963e+09
6,Nnew York Convention Center Corporation,1.382477e+09
7,BPP ST Owner LLC,1.360162e+09
8,Citifield,1.332865e+09
9,SL Green Realty Corporation,1.327638e+09


In [11]:
lpl[['Owner', 'Sq. Footage']]

,Owner,Sq. Footage
Rank,,
1,Riverbay Corporation,21257474.0
2,Trustees Of Columbia,10405555.0
3,ST-DIL LLC,8942176.0
4,"Starrett City, Inc.",8544670.0
5,Rochdale Village Inc,6996835.0
6,New York University,6370834.0
7,St. John's University,4687440.0
8,RCPI Landmark Properties LLC,4632189.0
9,New York-Presbyterian Hospital,4581586.0


In [20]:
# currently shows Columbia's property, as well a useless radio group buttion

from bokeh.io import output_file, show, vform, vplot, gridplot, hplot, curdoc, output_notebook
from bokeh.models import GMapPlot, GMapOptions, DataRange1d, ColumnDataSource, Circle, PanTool, WheelZoomTool, HoverTool, HBox, VBox, VBoxForm, BoxZoomTool
from bokeh.models.widgets import RadioButtonGroup, Slider, Select

addresses = literal_eval(wpl['Places'][1]) # for now just plot Columbia
map_options = GMapOptions(lat=40.6773, lng=-73.9857, map_type = "terrain", zoom=10)
# TODO: plot focus needs to be adjusted further to fix Bronx mapping.

GMap = GMapPlot(x_range = DataRange1d(),
                y_range = DataRange1d(),
                map_options = map_options ,
                title = "Eneumerate",
                plot_width=960,
                plot_height=600)

lats = []
longs = []
names = []
for address in addresses.keys():
    names.append(address)
    lats.append(addresses[address][0])
    longs.append(addresses[address][1])

Property = ColumnDataSource(data=dict(lat = lats, 
                                      lon = longs,
                                      name = names))

Address_Circle = Circle(x="lon", y="lat", size = 10)

GMap.add_glyph( Property, Address_Circle)
hover = HoverTool( tooltips=[(" ", "@name")])
GMap.add_tools(hover, PanTool(), WheelZoomTool(), BoxZoomTool())
GMap.logo = None
# GMap.toolbar_location = None

radio_button_group = RadioButtonGroup(labels=["Enumerate", "Value", "Square Footage"], active=0)

select = Select(options=list(mepl['Owner']))

def update(attrname, old, new):
    if radio_button_group.active == 0:
        df = mepl
    elif radio_button_group.active == 1:
        df = lpl
    else:
        df = wpl
    addresses = literal_eval(df['Places'][1])
    names = lats = longs = []
    for address in addresses.keys():
        names.append(address)
        lats.append(addresses[address][0])
        longs.append(addresses[address][1])
    Property.data = dict(
        lat = lats,
        lon = longs,
        name = names
    )

# for r, s in [radio_button_group, select]:
    
plot = vform(GMap, radio_button_group, select)

for w in [radio_button_group, select]:
    w.on_change('value', update)

curdoc().add_root(HBox(children=[radio_button_group, select, plot], width=800))

output_file("test.html")
# output_notebook()
show(plot)

In [13]:
from bokeh.models import TextInput, Select
from bokeh.plotting import Figure

In [14]:
# # Create Input controls
# reviews = Slider(title="Minimum number of reviews", value=80, start=10, end=300, step=10)
# min_year = Slider(title="Year released", start=1940, end=2014, value=1970, step=1)
# max_year = Slider(title="End Year released", start=1940, end=2014, value=2014, step=1)
# oscars = Slider(title="Minimum number of Oscar wins", start=0, end=4, value=0, step=1)
# boxoffice = Slider(title="Dollars at Box Office (millions)", start=0, end=800, value=0, step=1)
# # genre = Select(title="Genre", value="All",
# #                options=open(join(dirname(__file__), 'genres.txt')).read().split())
# director = TextInput(title="Director name contains")
# cast = TextInput(title="Cast names contains")
# # x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="Tomato Meter")
# # y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="Number of Reviews")

# # Create Column Data Source that will be used by the plot
# source = ColumnDataSource(data=dict(x=[], y=[], color=[], title=[], year=[], revenue=[]))

# hover = HoverTool(tooltips=[
#     ("Title","@title"),
#     ("Year", "@year"),
#     ("$", "@revenue")
# ])

# p = Figure(plot_height=600, plot_width=800, title="", toolbar_location=None, tools=[hover])
# p.circle(x="x", y="y", source=source, size=7, color="color", line_color=None, fill_alpha="alpha")

# # def select_movies():
# #     genre_val = genre.value
# #     director_val = director.value.strip()
# #     cast_val = cast.value.strip()
# #     selected = movies[
# #         (movies.Reviews >= reviews.value) &
# #         (movies.BoxOffice >= (boxoffice.value * 1e6)) &
# #         (movies.Year >= min_year.value) &
# #         (movies.Year <= max_year.value) &
# #         (movies.Oscars >= oscars.value)
# #     ]
# #     if (genre_val != "All"):
# #         selected = selected[selected.Genre.str.contains(genre_val)==True]
# #     if (director_val != ""):
# #         selected = selected[selected.Director.str.contains(director_val)==True]
# #     if (cast_val != ""):
# #         selected = selected[selected.Cast.str.contains(cast_val)==True]
# #     return selected

# def update(attrname, old, new):
#     df = select_movies()
#     x_name = axis_map[x_axis.value]
#     y_name = axis_map[y_axis.value]

#     p.xaxis.axis_label = x_axis.value
#     p.yaxis.axis_label = y_axis.value
#     p.title = "%d movies selected" % len(df)
#     source.data = dict(
#         x=df[x_name],
#         y=df[y_name],
#         color=df["color"],
#         title=df["Title"],
#         year=df["Year"],
#         revenue=df["revenue"],
#         alpha=df["alpha"],
#     )

# controls = [reviews, boxoffice, min_year, max_year, oscars, director, cast]
# # for control in controls:
# #     control.on_change('value', update)

# inputs = HBox(VBoxForm(*controls), width=300)

# # update(None, None, None) # initial load of the data

# curdoc().add_root(HBox(inputs, p, width=1100))

# output_file('test.html')

# show(p)

In [15]:
# ''' Present an interactive function explorer with slider widgets.
# Scrub the sliders to change the properties of the ``sin`` curve, or
# type into the title text box to update the title of the plot.
# Use the ``bokeh serve`` command to run the example by executing:
#     bokeh serve sliders.py
# at your command prompt. Then navigate to the URL
#     http://localhost:5006/sliders
# in your browser.
# '''
# import numpy as np

# from bokeh.plotting import Figure
# from bokeh.models import ColumnDataSource, HBox, VBoxForm
# from bokeh.models.widgets import Slider, TextInput
# from bokeh.io import curdoc

# # Set up data
# N = 200
# x = np.linspace(0, 4*np.pi, N)
# y = np.sin(x)
# source = ColumnDataSource(data=dict(x=x, y=y))


# # Set up plot
# plot = Figure(plot_height=400, plot_width=400, title="my sine wave",
#               tools="crosshair,pan,reset,resize,save,wheel_zoom",
#               x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

# plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


# # Set up widgets
# text = TextInput(title="title", value='my sine wave')
# offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
# amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0)
# phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
# freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1)


# # Set up callbacks
# def update_title(attrname, old, new):
#     plot.title = text.value

# text.on_change('value', update_title)

# def update_data(attrname, old, new):

#     # Get the current slider values
#     a = amplitude.value
#     b = offset.value
#     w = phase.value
#     k = freq.value

#     # Generate the new curve
#     x = np.linspace(0, 4*np.pi, N)
#     y = a*np.sin(k*x + w) + b

#     source.data = dict(x=x, y=y)

# for w in [offset, amplitude, phase, freq]:
#     w.on_change('value', update_data)


# # Set up layouts and add to document
# inputs = VBoxForm(children=[text, offset, amplitude, phase, freq])
# layout = HBox(children=[inputs, plot], width=800)


# curdoc().add_root(layout)

# output_file('test2.html')
# show(layout)